# Parse MultiXrank results

## Summary of the pipeline so far

MultiXrank is a Random Walk with Restart algorithm designed to explore multilayer networks. Starting from a seed node, it assigns scores to all nodes in the network with respect to the seed. Theses scores indicate how close a network node is to the seed.

We ran MultiXrank on a multilayer network composed of two layers:
- the **Rare-X layer** contains disease (Rare-X name format), patient, and symptom nodes (file `network/multiplex/RareX_layer/RareX_layer.tsv`).
All associations from the Rare-X layer are weighted:
    - edges between **diseases and patients**: weights = 1
    - edges between **patients and symptoms**:
        - normalized CSHQ scores for CSHQ symptoms
        - for other symptoms, we only put an edge if the patient presents the symptom, and theses edges are weighted 1

- the **Orphanet layer** contains disease (Orphanet name format) and phenotypes (Human Phenotye Onthology HPO) nodes (file `network/multiplex/Orphanet_layer/Orphanet_layer.tsv`).
All associations from the Orphanet layer are weighted:
    - edges between **diseases**: weights = 1
    - edges between **diseases and phenotypes**, weight is based on the association frequency from Orphanet:
        - obligate (100%) -> weight = 1
        - very frequent (99-80%) -> weight = 4/5
        - frequent (79-30%) -> weight = 3/5
        - occasional (29-5%) -> weight = 2/5
        - very rare (<4-1%) -> weight = 1/5
        - excluded (0%) -> weight = 0
    - edges between **phenotypes**: weight = 0.2
    
Our hypothesis is that MultiXrank can uncover previously unknown symptoms associated with rare diseases. Taking iteratively each Rare-X disease as a seed, we hypothesise that MultiXrank can highlight the symptoms that have a high score with respect to the seed disease but do not have similar phenotypes with also a high score. These uncorrelated symptoms/phenotypes might indicate new and unrecognized aspects of the disease's phenotype, potentially leading to valuable insights for diagnosis and treatment.

We run MultiXrank 27 times, using each time one of the 27 Rare-X diseases as seed. You can retreive the corresponding seed number in the `data/Diseases_names_and_seeds_numbering.tsv` file.

We obtain two result files containing the node scores (one file for each layer analysed) for each MultiXrank run. 

In this notebook, we parse MultiXrank results to create a summary file for each disease. 

In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
from pyhpo import Ontology
import numpy as np
import os

## Functions needed to reshape the results

### 1) Retreiving the phenotypes

First, we retreive the phenotypes from the Human Phenotype Onthology (HPO), stored in `en_product4.xml` file. 

In [2]:
tree = ET.parse("../data/en_product4.xml")
root = tree.getroot()

# initilize the Ontology ()
_ = Ontology()

### 2) Creating mapping between disease names and seed names

We create a mapping table between disease names and the seed numbers used in MultiXrank. We use the function `create_table_diseases_seeds()` to generate this mapping table. 

In [3]:
def create_table_diseases_seeds(mapping_file: str, table_name: str) -> dict:
    """Function that generates a mapping table of disease names and the 
    numbers used in MultiXrank to idenfity diseases

    Args:
        mapping_file (str): name of the mapping file
        table_name (str): name of the output table

    Returns:
        dict: a dictionary of correspondances between rare-x diseases names,
        orphanet diseases names and seed numbers used in MultiXrank
    """
    dico_diseases_seeds = dict()
    df_mapping_file = pd.read_csv(mapping_file, sep=";", header=0)
    df_table = pd.DataFrame(columns=["RARE-X", "ORPHANET", "SEED NUMBER"])
    df_table["RARE-X"] = df_mapping_file["Rx"]
    diseases = df_table["RARE-X"].tolist()
    df_table["ORPHANET"] = df_mapping_file["Orphanet"]
    seed_numbers = [i for i in range(1, 28)]
    df_table["SEED NUMBER"] = seed_numbers
    df_table.to_csv(table_name, sep="\t", header=True, index=False)
    for disease, seed in zip(diseases, seed_numbers):
        dico_diseases_seeds[disease] = seed
    return dico_diseases_seeds

dico_diseases_seeds = create_table_diseases_seeds(mapping_file="../data/Diseases_Rx_orpha_corres.csv", table_name="../data/Diseases_names_and_seeds_numbering.tsv")

### 3) Retrieving Orphanet names from Orphacodes

The `find_orpha_name()` function gives the disease Orphanet name for a given Orphanet code. 

In [4]:
def find_orpha_name(orpha_code: str) -> str:
    """Function that returns the orphanet
    name of a disease given its orphanet
    code

    Args:
        orpha_code (str): orphanet code of 
        the disease

    Returns:
        str: the orphanet name of the disease
    """
    for disorder in root.iter('Disorder'):
        orpha_code_in_tree = disorder.find('OrphaCode').text
        orpha_name = disorder.find('Name').text
        if orpha_code_in_tree == orpha_code:
            return orpha_name

### 4) Reshaping the results with the `create_results_table` function

The `create_results_table()` function reads the MultiXrank results files. Note that for each run done on a given seed, MultiXrank output two score files, one for each layer in the multilayer network (in our case, one for the Rare-X layer and one for the Orphanet layer).
- MultiXrank results are stored in the `multiplex_RareX_layer.tsv` files for the Rare-X layer.
- MultiXrank results are stored in the `multiplex_Orphanet_layer.tsv` files for the Orphanet layer.

These files are large, and reading the entire files take time. Therefore, we only load the first 1000 lines of each file. You can change this number using the `input_nrow` parameter.

The function adds the Orphanet disease name and the phenotype name for each corresponding Orphacode and phenotype that are in the Orphanet layer.  It also removes "patient scores" from the results of the Rare-X layer, since we are interested only in "symptom scores".

We select the top 20 scores, to simplify the results analysis. You can change this top number using the `output_top` parameter. 

Finally, the function concatenates the top nodes from each layer into a single `tsv` file. The output directory name can be set up using the `resultsdir` parameter. 

In [5]:
def create_results_table(dico_diseases_seeds: dict, input_nrow: int, output_top: int, outdir: str, resultsdir: str) -> None:
    """Function that generates for each MutliXrank output = 
    each rarex disease, a results file that recapitulates/concatenates 
    all the scores of the 2 layers in a single file

    Args:
        dico_diseases_seeds (dict): a dictionary of the rarex 
        diseases and their seed numbers used in MultiXrank
        input_nrow (int): number of lines to read in MultiXrank outputs
        output_top (int): number of top results to select for output
        outdir (str): MultiXrank output directory
        resultsdir (str): results output directory

    Remark: read full output file could take time
    """
    
    os.makedirs(f"../results_MultiXrank/{resultsdir}/", exist_ok=True)
    for disease, seed in dico_diseases_seeds.items():
        # Read layer 1 (rarex) output: no terms description to add
        # because this layer contains Rare-X disease names, symptomes
        # names and patients IDs
        multiplex_layer1 = pd.read_csv(f"../results_MultiXrank/{outdir}/output_{seed}/multiplex_RareX_layer.tsv", header=0, sep="\t", nrows=input_nrow)
        multiplex_layer1.rename(columns={"multiplex": "layer"}, inplace=True)
        multiplex_1_selected = multiplex_layer1[multiplex_layer1['node'].str.contains('[A-Za-z]+', regex=True)]
        
        # Read layer 2 (orpha-hpo) output
        multiplex_layer2 = pd.read_csv(f"../results_MultiXrank/{outdir}/output_{seed}/multiplex_Orphanet_layer.tsv", header=0, sep="\t", nrows=input_nrow)
        multiplex_layer2.rename(columns={"multiplex": "layer"}, inplace=True)
        # get the nodes into a list
        nodes_layer2 = multiplex_layer2[multiplex_layer2.columns[1]].to_list()
        # initialize empty list to store the descriptions (orpha names and phenotyes names) for each node
        list_description_layer2 = list()
        # browse nodes in mutliplex 2 to add description
        for term in nodes_layer2:
            if term[:5] == "ORPHA":
                orpha_code = term[6:]
                orpha_name = find_orpha_name(orpha_code=orpha_code)
                list_description_layer2.append(orpha_name)
            elif term[:2] == "HP":
                try:
                    hpo_phenotype = Ontology.get_hpo_object(term)
                    list_description_layer2.extend([str(hpo_phenotype)[13:]])
                # if there is no match of HPO phenotype name
                except RuntimeError:
                    list_description_layer2.append("None")
        # check that the description list and the dataframe have the same length !
        assert len(list_description_layer2) == len(multiplex_layer2.index)
        # create new description columns for the terms
        description_layer2 = pd.DataFrame(list_description_layer2, columns=['description'])
        # create new dataframe for layer 2 containing the ranking of the nodes + their description (orpha names and phenotypes names)
        multiplex_2_with_description = pd.concat([multiplex_layer2.reindex(range(len(multiplex_layer2))), description_layer2.reindex(range(len(multiplex_layer2)))], axis=1)
        
        # Select top of results
        multiplex_1_head = multiplex_1_selected.reset_index(drop=True).head(21)
        multiplex_1_head["empty"] = ""
        multiplex_2_head = multiplex_2_with_description.reset_index(drop=True).head(21)
        
        # concatenate the two dataframes and generate table output
        table_results = pd.concat([multiplex_1_head, multiplex_2_head], axis=1)
        table_results.to_csv(f"../results_MultiXrank/{resultsdir}/results_disease_{seed}.tsv", sep="\t", header=True, index=False)


In [6]:
## Parameters
outdir = "output"
resultsdir = "results"
input_nrow = 1000
output_top = 21

## Results integration
create_results_table(dico_diseases_seeds=dico_diseases_seeds,
                     input_nrow=input_nrow, 
                     output_top=output_top,
                     outdir=outdir, 
                     resultsdir=resultsdir)